<a href="https://colab.research.google.com/github/flipfelly/Streaming-com-Kinesis/blob/main/Produtor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aplicações em Streaming com AWS Kinesis

Este código utiliza a biblioteca **boto3** para interagir com o serviço **Amazon Kinesis**, da AWS, que permite coletar, processar e analisar dados em tempo real. Inicialmente, é criado um cliente Kinesis com as credenciais de acesso e a região especificadas. Em seguida, é definido um registro de dados no formato de dicionário Python, contendo informações de um vendedor, como o **ID** e o **nome**.

Kinesis não é um serviço gratuito então podem haver crobanças.

---

## Requisitos
- Conta AWS configurada.
- Um fluxo de dados criado no **Kinesis**.

---

## Como criar um fluxo de dados no Kinesis:
1. Na tela inicial da AWS, pesquise por **Kinesis** e acesse sua página.
2. Certifique-se de ter a opção **Kinesis Data Streams** selecionada.
3. Clique em **Criar fluxo de dados**.
4. Nomeie seu fluxo de dados (este nome será utilizado no código).
5. Em **Modo de capacidade**, selecione **Sob demanda** (a AWS ajustará automaticamente as necessidades).
6. Clique em **Criar fluxo de dados**.

---

## Como encontrar a `region_name`:
1. Clique no nome do fluxo de dados recém-criado na página do **Kinesis**.
2. Em **Resumo do fluxo de dados**, localize o **ARN**.  
   Você encontrará algo como:  
   arn:aws:kinesis:`us-east-1`:356397688932:stream/stream1.  
3. Copie apenas a parte destacada, como **us-east-1**.
   > **Atenção:** O código varia de acordo com a região onde o fluxo foi criado. Não utilize o código do exemplo.

---

## Como encontrar o `StreamName`:
1. O **StreamName** é o nome que você deu ao criar o fluxo de dados.  
   Caso tenha esquecido, siga os passos abaixo:
2. Na página do **Kinesis**, acesse **Streams de dados**.
3. Copie o nome do fluxo criado.

---
## Como obter suas `credenciais AWS` (aws_access_key_id e aws_secret_access_key):
1. No canto superior direito da tela, encontre o **nome do usuário**.  
2. Clique na **setinha** ao lado do nome.  
3. Selecione **Credenciais de Segurança**.  
4. Localize a seção **Chaves de Acesso**.  
5. Crie uma nova chave de acesso.  
6. Salve o arquivo com a chave ou copie as informações (id e chave).

⚠️ **Atenção**: Você só poderá visualizar a chave de acesso no momento da criação, tendo acesso somente ao ID. Guarde-a em local seguro!



In [1]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00


# Explicação das Variáveis `cliente` e `resposta`

## Variável `cliente`
A variável **`cliente`** é uma instância do cliente AWS Kinesis criada pelo **boto3**. Essa variável é utilizada para interagir com o serviço de streaming de dados oferecido pela Amazon.

### Componentes:
1. **`boto3.client('kinesis')`**:
   - Cria um cliente para o serviço **Amazon Kinesis**.

2. **`aws_access_key_id` e `aws_secret_access_key`**:
   - Credenciais que autenticam o acesso à conta AWS, permitindo ao cliente executar operações no Kinesis.
   - **Atenção**: Essas credenciais são sensíveis e devem ser protegidas para evitar acesso não autorizado.

3. **`region_name`**:
   - Especifica a região onde o fluxo de dados do Kinesis foi configurado (exemplo: **`us-east-1`**).

---

## Variável `resposta`


### Uso do `.json` e `PartitionKey`
- O **`.json`** é utilizado para converter objetos Python em um formato compatível com o Kinesis, garantindo que os dados sejam enviados de maneira estruturada e legível.
  
- A **`PartitionKey`** é usada para distribuir os dados entre shards no fluxo, aplicando uma função de hash. No nosso caso, em um único shard, não faça diferença, ela é fundamental para equilibrar a carga entre múltiplos shards.

### **`ShardId`**
- **`ShardId`** identifica o **shard** (subdivisão do fluxo de dados) onde o registro foi armazenado. Shards são responsáveis por particionar os dados dentro do fluxo. ESSE REGISTRO SERÁ UTILIZADO NO NOTEBOOK CONSUMIDOR



In [10]:
import boto3
import json

cliente = boto3.client('kinesis', aws_access_key_id = 'id da chave de acesso', aws_secret_access_key = "chave de acesso", region_name='us-east-1')

registro = {'idvendedor': '231', 'nome':'Pedro'} #dado a ser registrado

resposta = cliente.put_record(StreamName='nome do stream criado', Data=json.dumps(registro), PartitionKey='02') #PartitionKey qualquer pois não influencia na realidade que estamos aplicando

print(resposta)

{'ShardId': 'shardId-000000000002', 'SequenceNumber': '49659682377937857660418786304854671273638241077525741602', 'ResponseMetadata': {'RequestId': 'e4f11055-2472-8e05-84f5-4c122fc0210c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e4f11055-2472-8e05-84f5-4c122fc0210c', 'x-amz-id-2': 'sbrD+1KLeY3D11hLjY5rZo1HpWuGH7N5PvTdugUYSLBAGSu/3pcjt0JpMiIVlGTGNwzt2yu6Cnl5K1Byi0A13f6AZFXpVl7y', 'date': 'Fri, 17 Jan 2025 13:24:23 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


# Consumidor: Notebook para Receber Dados em Streaming

Este notebook será utilizado para simular outra instância (ou máquina virtual) que irá consumir os dados enviados pelo notebook **Produtor**. Ele estará continuamente ouvindo os dados enviados pelo **Kinesis Data Streams**.

---

## Etapas para Configurar o Notebook Consumidor:

1. **Criar um novo notebook:**
   - Este notebook representará a instância Consumidora.

2. **Instalar o boto3:**
   - Execute o seguinte comando no notebook:  
     ```python
     !pip install boto3
     ```

3. **Inserir o código a seguir no novo notebook:**
   ```python
   import boto3

   cliente = boto3.client(
       'kinesis',
       aws_access_key_id='Id da chave de acesso',
       aws_secret_access_key='chave de acesso',
       region_name='regiao do serviço aws'
   )

   shard = cliente.get_shard_iterator(
       StreamName='Nome do fluxo criado no kinesis',
       ShardId='Shard ID retornado pela célula do notebook Produtor',
       ShardIteratorType='LATEST'
   )['ShardIterator']

   while shard is not None:
       resultado = cliente.get_records(ShardIterator=shard)
       registros = resultado['Records']
       shard = resultado["NextShardIterator"]

       for registro in registros:
           print(registro["SequenceNumber"])
           print(registro["ApproximateArrivalTimestamp"])
           print(registro["PartitionKey"])
           print(registro["Data"])
      ```
---

## Observações:

### Manter o notebook rodando:
- A célula **não irá parar automaticamente**, a menos que você:
  - Encerre manualmente o processo (via **Keyboard Interrupt**).
  - Ou exclua o fluxo de dados no **AWS Kinesis**.

### Teste com o Produtor:
- Após configurar e executar o código no notebook **Consumidor** (não interrompa a célula), volte para o notebook **Produtor** e:
  1. Modifique os registros (como **ID** ou **Nome**).
  2. Execute o notebook Produtor a cada modificação.
  3. Verifique se os resultados aparecem corretamente no notebook Consumidor.

### Resultados no Consumidor:
- O notebook Consumidor imprimirá as seguintes informações para cada registro recebido:
  - **SequenceNumber**: Número sequencial do registro.
  - **ApproximateArrivalTimestamp**: Estampa de tempo aproximada da chegada do registro.
  - **PartitionKey**: Chave de partição utilizada.
  - **Data**: Dados enviados pelo Produtor.

  
  # O NOTEBOOK CONSUMIDOR TAMBEM ESTÁ DISPONÍVEL PARA ACESSO COM INSTRUÇÕES
